In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


**3

For N labels in multi-label classification, it doesn't really matter whether you sum the loss for each class, or whether you compute the average loss using tf.reduce_mean: the gradient would point in the same direction.

However, if you divide the sum by N (this is what averaging essentially is), this will influence the learning rate at the end of the day. If you are not sure how many labels in the multi-label classification task there will be, it might be easier to use tf.reduce_mean, because you wouldn't have to readjust the weight of this loss component compared to other components of the loss, and you wouldn't have to adjust the learning rate in the number N of labels changes.

shareimprove this answer**

In [2]:
# https://stackoverflow.com/questions/40198364/how-can-i-implement-a-weighted-cross-entropy-loss-in-tensorflow-using-sparse-sof

In [3]:
# https://stackoverflow.com/questions/35155655/loss-function-for-class-imbalanced-binary-classifier-in-tensor-flow/35168022#35168022

In [4]:
import  tensorflow as tf
import numpy as np

np.random.seed(123)
sess = tf.InteractiveSession()

# let's say we have the logits and labels of a batch of size 6 with 5 classes
logits = tf.constant(np.random.randint(0, 10, 30).reshape(6, 5), dtype=tf.float32)
labels = tf.constant(np.random.randint(0, 5, 6), dtype=tf.int32)



In [5]:

logits.eval()

array([[2., 2., 6., 1., 3.],
       [9., 6., 1., 0., 1.],
       [9., 0., 0., 9., 3.],
       [4., 0., 0., 4., 1.],
       [7., 3., 2., 4., 7.],
       [2., 4., 8., 0., 7.]], dtype=float32)

In [6]:
labels.eval ()

array([1, 3, 4, 4, 4, 1], dtype=int32)

In [7]:
# specify some class weightings
class_weights = tf.constant([0.3, 0.1, 0.2, 0.3, 0.1])

# specify the weights for each sample in the batch (without having to compute the onehot label matrix)


In [8]:
weights = tf.gather(class_weights, labels)

# compute the loss


In [9]:
weights.eval()

array([0.1, 0.3, 0.1, 0.1, 0.1, 0.1], dtype=float32)

In [10]:
tf.losses.sparse_softmax_cross_entropy(labels, logits, weights).eval()

Instructions for updating:
Use tf.cast instead.


0.77875894

In [11]:
x=tf.ones(shape=[3,4])
w=tf.constant([0.1,0.3,0.4,0.5])

In [12]:
(x*w).eval()  #so multiplication operation gets broadcased

array([[0.1, 0.3, 0.4, 0.5],
       [0.1, 0.3, 0.4, 0.5],
       [0.1, 0.3, 0.4, 0.5]], dtype=float32)

In [13]:
(x+w).eval()  # so addition operation gets broadcased

array([[1.1, 1.3, 1.4, 1.5],
       [1.1, 1.3, 1.4, 1.5],
       [1.1, 1.3, 1.4, 1.5]], dtype=float32)

In [14]:
#binary crossentropy
x=tf.constant(np.random.uniform(0,1,size=[3,4]),dtype=tf.float32)

In [15]:
x.eval() # assume these are probabilities predicted by model

array([[0.6547213 , 0.37380144, 0.23451288, 0.98799527],
       [0.765996  , 0.7770044 , 0.02798196, 0.17390652],
       [0.15408224, 0.07708648, 0.8898657 , 0.7503787 ]], dtype=float32)

In [16]:
y=tf.constant([1,0,0,1],dtype=tf.float32)

In [17]:
y1=tf.stack([y,y,y])

In [18]:
y1.eval()

array([[1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.]], dtype=float32)

In [19]:
loss=tf.nn.sigmoid_cross_entropy_with_logits(labels=y1,logits=x)

In [20]:
loss.eval()  #losses for different classes class 1 , class2, class 3, class 4

array([[0.4184385 , 0.89741313, 0.8172624 , 0.31650445],
       [0.38176697, 1.1552911 , 0.707236  , 0.6099696 ],
       [0.61907077, 0.7324331 , 1.2339594 , 0.38674954]], dtype=float32)

In [21]:
x.eval()

array([[0.6547213 , 0.37380144, 0.23451288, 0.98799527],
       [0.765996  , 0.7770044 , 0.02798196, 0.17390652],
       [0.15408224, 0.07708648, 0.8898657 , 0.7503787 ]], dtype=float32)

In [22]:
# if class2 is imbalanced
# https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras

In [23]:
#let say 
w=tf.constant([1,2,1,2],dtype=tf.float32)
w=tf.reshape(w,shape=(1,4))

In [24]:
loss1=loss*w

In [25]:
loss1.eval()

array([[0.4184385 , 1.7948263 , 0.8172624 , 0.6330089 ],
       [0.38176697, 2.3105822 , 0.707236  , 1.2199392 ],
       [0.61907077, 1.4648662 , 1.2339594 , 0.7734991 ]], dtype=float32)

In [26]:
loss.eval() #so we have increased loss for skewed label classes which will force the model to spend more time on classificatin of class 2

array([[0.4184385 , 0.89741313, 0.8172624 , 0.31650445],
       [0.38176697, 1.1552911 , 0.707236  , 0.6099696 ],
       [0.61907077, 0.7324331 , 1.2339594 , 0.38674954]], dtype=float32)

In [27]:
netloss=tf.reduce_mean(loss1)

In [28]:
netloss.eval()

1.0312047